In [2]:
import numpy as np 
import pandas as pd
import pyarrow.parquet as pq

In [3]:
df=pq.read_table('train.parquet').to_pandas()


In [4]:
df.head()

,Date,X1,X2,X3,X4,X5,target
0,2020-12-16,1.518921,5.463154,1.0,2.718282,2.890372,0
1,2020-12-16,1.546509,5.458010,1.0,2.718282,2.833213,1
2,2020-12-16,1.645427,5.456560,1.0,7.389056,2.890372,1
3,2020-12-16,1.652022,5.458479,1.0,2.718282,2.890372,1
4,2020-12-16,1.695538,5.466709,1.0,2.718282,2.890372,0


In [5]:
df.tail()

,Date,X1,X2,X3,X4,X5,target
1639419,2024-11-26,1.0,5.482346,1.0,1.0,0.0,0
1639420,2024-11-26,1.0,5.486580,1.0,1.0,0.0,0
1639421,2024-11-26,1.0,5.484797,1.0,1.0,0.0,0
1639422,2024-11-26,1.0,5.483551,1.0,1.0,0.0,0
1639423,2024-11-26,1.0,5.482928,1.0,1.0,0.0,0


In [6]:
df.isnull().sum()

Date      0
X1        0
X2        0
X3        0
X4        0
X5        0
target    0
dtype: int64

In [7]:
df["target"].value_counts(normalize=True)


target
0    0.991437
1    0.008563
Name: proportion, dtype: float64

In [8]:
df.nunique()


Date      1432
X1         526
X2        2812
X3          88
X4          81
X5          32
target       2
dtype: int64

In [10]:
df=df.sort_values("Date")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1639424 entries, 0 to 1623460
Data columns (total 7 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   Date    1639424 non-null  datetime64[ns]
 1   X1      1639424 non-null  float64       
 2   X2      1639424 non-null  float64       
 3   X3      1639424 non-null  float64       
 4   X4      1639424 non-null  float64       
 5   X5      1639424 non-null  float64       
 6   target  1639424 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 100.1+ MB


In [12]:
df.describe()
df['target'].value_counts(normalize=True)
df.nunique()

Date      1432
X1         526
X2        2812
X3          88
X4          81
X5          32
target       2
dtype: int64

In [13]:
df['target'].value_counts(normalize=True)

target
0    0.991437
1    0.008563
Name: proportion, dtype: float64

In [15]:
df['month']=df['Date'].dt.month
df['day']=df['Date'].dt.day
df['weekday']=df['Date'].dt.weekday

In [16]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")


In [76]:
print(df["target"].dtype)

int64


In [77]:
split_date = df["Date"].quantile(0.8)


In [78]:
train = df[df["Date"] <= split_date]
test  = df[df["Date"] > split_date]


In [79]:
X_train = train.drop(["target", "Date"], axis=1)
y_train = train["target"]

X_test = test.drop(["target", "Date"], axis=1)
y_test = test["target"]


In [80]:
print(train["Date"].min(), "to", train["Date"].max())
print(test["Date"].min(), "to", test["Date"].max())


2020-12-16 00:00:00 to 2024-02-05 00:00:00
2024-02-06 00:00:00 to 2024-12-11 00:00:00


In [81]:
train.info()
train['target'].value_counts(normalize=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1311571 entries, 0 to 1409886
Data columns (total 10 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   Date     1311571 non-null  datetime64[ns]
 1   X1       1311571 non-null  float64       
 2   X2       1311571 non-null  float64       
 3   X3       1311571 non-null  float64       
 4   X4       1311571 non-null  float64       
 5   X5       1311571 non-null  float64       
 6   target   1311571 non-null  int64         
 7   month    1311571 non-null  int32         
 8   day      1311571 non-null  int32         
 9   weekday  1311571 non-null  int32         
dtypes: datetime64[ns](1), float64(5), int32(3), int64(1)
memory usage: 95.1 MB


target
0    0.990315
1    0.009685
Name: proportion, dtype: float64

In [82]:
X_train = train.drop(["target", "Date"], axis=1)
y_train = train["target"]


In [83]:
train.loc[:, "target"] = train["target"].astype(int)


In [84]:
X_train = train.drop(["target", "Date"], axis=1)
y_train = train["target"]
#drop target cause we dont want data to see the target that we will be training it on
#axi=1 means remove column , axis=0 means remove row

In [85]:


y_train = train["target"]


In [86]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [87]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200,
    class_weight="balanced",
    random_state=42
)


In [88]:
model.fit(X_train, y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [89]:

from sklearn.metrics import classification_report


In [90]:
from sklearn.metrics import classification_report
y_pred=model.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1298869
           1       1.00      1.00      1.00     12702

    accuracy                           1.00   1311571
   macro avg       1.00      1.00      1.00   1311571
weighted avg       1.00      1.00      1.00   1311571



In [91]:
df["target"] = df["target"].astype(int)


In [92]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    326517
           1       0.83      0.40      0.54      1336

    accuracy                           1.00    327853
   macro avg       0.91      0.70      0.77    327853
weighted avg       1.00      1.00      1.00    327853



In [93]:

y_prob = model.predict_proba(X_test)[:, 1]

y_pred_new = (y_prob > 0.1).astype(int)

print(classification_report(y_test, y_pred_new))


              precision    recall  f1-score   support

           0       1.00      0.99      1.00    326517
           1       0.31      0.78      0.45      1336

    accuracy                           0.99    327853
   macro avg       0.66      0.88      0.72    327853
weighted avg       1.00      0.99      0.99    327853



In [94]:
import numpy as np
from sklearn.metrics import f1_score

y_prob = model.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.01, 0.5, 0.02)

for t in thresholds:
    y_pred_temp = (y_prob > t).astype(int)
    f1 = f1_score(y_test, y_pred_temp)
    print(f"Threshold: {t:.2f} | F1: {f1:.4f}")


Threshold: 0.01 | F1: 0.1526
Threshold: 0.03 | F1: 0.2517
Threshold: 0.05 | F1: 0.3084
Threshold: 0.07 | F1: 0.3657
Threshold: 0.09 | F1: 0.4155
Threshold: 0.11 | F1: 0.4558
Threshold: 0.13 | F1: 0.4944
Threshold: 0.15 | F1: 0.5294
Threshold: 0.17 | F1: 0.5476
Threshold: 0.19 | F1: 0.5680
Threshold: 0.21 | F1: 0.5830
Threshold: 0.23 | F1: 0.5954
Threshold: 0.25 | F1: 0.6025
Threshold: 0.27 | F1: 0.6062
Threshold: 0.29 | F1: 0.6045
Threshold: 0.31 | F1: 0.6065
Threshold: 0.33 | F1: 0.5928
Threshold: 0.35 | F1: 0.5901
Threshold: 0.37 | F1: 0.5853
Threshold: 0.39 | F1: 0.5764
Threshold: 0.41 | F1: 0.5718
Threshold: 0.43 | F1: 0.5639
Threshold: 0.45 | F1: 0.5532
Threshold: 0.47 | F1: 0.5513
Threshold: 0.49 | F1: 0.5436


In [95]:
pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 1.2 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.8 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [96]:
from lightgbm import LGBMClassifier


In [97]:
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()

scale_pos_weight = neg / pos
print(scale_pos_weight)


102.25704613446702


In [98]:
model_lgb = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=-1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)


In [99]:
model_lgb.fit(X_train, y_train)


[LightGBM] [Info] Number of positive: 12702, number of negative: 1298869
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017818 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 741
[LightGBM] [Info] Number of data points in the train set: 1311571, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.009685 -> initscore=-4.627490
[LightGBM] [Info] Start training from score -4.627490


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.05
,n_estimators,200
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [100]:
from sklearn.metrics import classification_report

y_prob = model_lgb.predict_proba(X_test)[:, 1]
y_pred = (y_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       1.00      0.97      0.98    326517
           1       0.11      0.96      0.19      1336

    accuracy                           0.97    327853
   macro avg       0.55      0.97      0.59    327853
weighted avg       1.00      0.97      0.98    327853



In [102]:
import numpy as np
from sklearn.metrics import f1_score

thresholds = np.arange(0.05, 0.6, 0.02)

for t in thresholds:
    y_temp = (y_prob > t).astype(int)
    print(f"Threshold {t:.2f} | F1: {f1_score(y_test, y_temp):.4f}")


Threshold 0.05 | F1: 0.0553
Threshold 0.07 | F1: 0.0660
Threshold 0.09 | F1: 0.0755
Threshold 0.11 | F1: 0.0837
Threshold 0.13 | F1: 0.0914
Threshold 0.15 | F1: 0.0989
Threshold 0.17 | F1: 0.1063
Threshold 0.19 | F1: 0.1129
Threshold 0.21 | F1: 0.1189
Threshold 0.23 | F1: 0.1240
Threshold 0.25 | F1: 0.1291
Threshold 0.27 | F1: 0.1342
Threshold 0.29 | F1: 0.1388
Threshold 0.31 | F1: 0.1431
Threshold 0.33 | F1: 0.1471
Threshold 0.35 | F1: 0.1514
Threshold 0.37 | F1: 0.1562
Threshold 0.39 | F1: 0.1619
Threshold 0.41 | F1: 0.1667
Threshold 0.43 | F1: 0.1729
Threshold 0.45 | F1: 0.1785
Threshold 0.47 | F1: 0.1838
Threshold 0.49 | F1: 0.1897
Threshold 0.51 | F1: 0.1951
Threshold 0.53 | F1: 0.2012
Threshold 0.55 | F1: 0.2070
Threshold 0.57 | F1: 0.2144
Threshold 0.59 | F1: 0.2221


Model comparision

In [3]:

pip install xgboost


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from xgboost import XGBClassifier

neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale = neg / pos

model_xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    scale_pos_weight=scale,
    random_state=42,
    n_jobs=-1,
    eval_metric="logloss"
)

model_xgb.fit(X_train, y_train)


NameError: name 'y_train' is not defined